In [35]:
import pandas as pd, os

# Path to the Excel file you just downloaded
EXCEL_PATH = os.path.join(os.pardir, 'data', 'full_monthly', 'urban_water.xlsx')

# Read the first sheet (it holds the supplier-month data)
raw = pd.read_excel(EXCEL_PATH, sheet_name=0)

print(raw.shape)          # rows, columns
raw.head()

(42469, 29)


,supplier_name,public_water_system_id,reporting_month,county,hydrologic_region,climate_zone,total_population_served,reference_2014_population,county_under_drought_declaration,water_shortage_contingency_stage_invoked,...,reported_preliminary_commercial_industrial_and_institutional_water,reported_final_commercial_industrial_and_institutional_water,reported_recycled_water,reported_non_revenue_water,calculated_total_potable_water_production_gallons_ag_excluded,calculated_total_potable_water_production_gallons_2013_ag_excluded,calculated_commercial_agricultural_water_gallons,calculated_commercial_agricultural_water_gallons_2013,calculated_r_gpcd,qualification
0,East Bay Municipal Utilities District,CA0110005,2023-04-15,"Alameda,Contra Costa",San Francisco Bay,3,1440000,1379000,NaN,NaN,...,NaN,NaN,NaN,NaN,4.026000e+09,5.418000e+09,NaN,0.0,58.7125,1. To provide context‚Ä¶the average R-GPCD for...
1,East Bay Municipal Utilities District,CA0110005,2023-03-15,"Alameda,Contra Costa",San Francisco Bay,3,1440000,1379000,Yes,1,...,739.0,NaN,NaN,NaN,3.672000e+09,4.928000e+09,NaN,0.0,47.7097,1. To provide context‚Ä¶the average R-GPCD for...
2,East Bay Municipal Utilities District,CA0110005,2023-02-15,"Alameda,Contra Costa",San Francisco Bay,3,1440000,1379000,Yes,Stage 2,...,767.0,NaN,NaN,NaN,3.358000e+09,4.016000e+09,NaN,0.0,44.9732,1. To provide context‚Ä¶the average R-GPCD for...
3,East Bay Municipal Utilities District,CA0110005,2023-01-15,"Alameda,Contra Costa",San Francisco Bay,3,1440000,1379000,Yes,Stage 2,...,939.0,NaN,NaN,NaN,3.696000e+09,4.099000e+09,NaN,0.0,48.0215,1. To provide context‚Ä¶the average R-GPCD for...
4,East Bay Municipal Utilities District,CA0110005,2022-12-15,"Alameda,Contra Costa",San Francisco Bay,3,1430000,1379000,Yes,Stage 2,...,816.0,NaN,NaN,NaN,3.722000e+09,4.772000e+09,NaN,0.0,50.3767,1. To provide context‚Ä¶the average R-GPCD for...


In [36]:
print(raw.columns.tolist())

['supplier_name', 'public_water_system_id', 'reporting_month', 'county', 'hydrologic_region', 'climate_zone', 'total_population_served', 'reference_2014_population', 'county_under_drought_declaration', 'water_shortage_contingency_stage_invoked', 'water_shortage_level_indicator', 'dwr_state_standard_level_corresponding_to_stage', 'water_production_units', 'reported_preliminary_total_potable_water_production', 'reported_final_total_potable_water_production', 'preliminary_percent_residential_use', 'final_percent_residential_use', 'reported_preliminary_commercial_agricultural_water', 'reported_final_commercial_agricultural_water', 'reported_preliminary_commercial_industrial_and_institutional_water', 'reported_final_commercial_industrial_and_institutional_water', 'reported_recycled_water', 'reported_non_revenue_water', 'calculated_total_potable_water_production_gallons_ag_excluded', 'calculated_total_potable_water_production_gallons_2013_ag_excluded', 'calculated_commercial_agricultural_wat

In [37]:
# Step 3 — standardise the column names (1 small cell)
# ── Clean header names ──────────────────────────────────────────
raw.columns = (
    raw.columns
        .str.strip()                         # remove leading/trailing spaces
        .str.lower()                         # all lower-case
        .str.replace(r'\s+', '_', regex=True)# spaces → underscores
)

print("First 20 columns")
print(raw.columns.tolist()[:20])

First 20 columns
['supplier_name', 'public_water_system_id', 'reporting_month', 'county', 'hydrologic_region', 'climate_zone', 'total_population_served', 'reference_2014_population', 'county_under_drought_declaration', 'water_shortage_contingency_stage_invoked', 'water_shortage_level_indicator', 'dwr_state_standard_level_corresponding_to_stage', 'water_production_units', 'reported_preliminary_total_potable_water_production', 'reported_final_total_potable_water_production', 'preliminary_percent_residential_use', 'final_percent_residential_use', 'reported_preliminary_commercial_agricultural_water', 'reported_final_commercial_agricultural_water', 'reported_preliminary_commercial_industrial_and_institutional_water']


### Recreate a compliance flag
1. Compliant if percent_saved ≥ state_standard_percent
2. Non-Compliant if percent_saved < state_standard_percent
3. “Not Required” months have state_standard_percent == 0 and were exempt; you can keep or drop them.